In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px  # plotly>=4.8.1
import plotly.graph_objects as go


In [3]:
df = pd.read_csv("data/repositories_core.csv")
# df = pd.DataFrame(df)
# df['keywords'] = str(df['keywords'])

### Keywords
___________________________________
Here we'll create a dataset that groups all details

In [4]:
#split publishers
keywords = df.set_index(['publisher','publisher_categories','author(s)', 'title', 'publicationYear']).apply(lambda x: x.astype(str).str.split(',').explode()).reset_index()
keywords = keywords.set_index(['publisher','publisher_categories','author(s)', 'title', 'publicationYear']).apply(lambda x: x.astype(str).str.split(';').explode()).reset_index()
keywords = keywords.drop_duplicates()
#clean up publisher names
keywords = keywords[keywords["keywords"].str.contains(".pdf") == False] 
keywords = keywords[keywords["keywords"].str.contains("http://") == False] 
keywords = keywords[keywords["keywords"].str.contains("rochester") == False] 
keywords = keywords[keywords["keywords"].str.contains("0") == False] 
keywords = keywords[keywords["keywords"].str.contains("erp") == False] 
keywords = keywords[keywords["keywords"].str.contains("clic") == False] 
keywords = keywords[keywords["keywords"].str.contains("52c22") == False] 
keywords = keywords[keywords["keywords"].str.contains("65f25") == False] 
keywords = keywords[keywords["keywords"].str.contains("firn") == False] 
keywords = keywords[keywords["keywords"].str.contains("summit") == False] 
keywords = keywords[keywords["keywords"].str.contains("eeg") == False] 
keywords = keywords[keywords["keywords"].str.contains("trusts") == False] 
keywords = keywords[keywords["keywords"].str.contains("de") == False] 
keywords = keywords[keywords["keywords"].str.contains("msc") == False] 
keywords = keywords[keywords["keywords"].str.contains("research") == False] 
keywords = keywords[keywords["keywords"].str.contains("arxiv") == False] 
keywords = keywords[keywords["keywords"].str.contains("ctsa") == False] 



# get every keyword an author used
keywords['tot_appearances'] = keywords.groupby('keywords')['keywords'].transform('count')
# get every year an author published something
keywords['tot_pubYrs'] = keywords.groupby(['keywords'])['publicationYear'].transform('nunique')
# get every author associated with a keyword
keywords['tot_authors'] = keywords.groupby(['keywords'])['author(s)'].transform('nunique')
# get every publisher affiliated with the author
keywords['tot_publishers'] = keywords.groupby(['keywords'])['publisher'].transform('nunique').astype(str)
# keywords['keywords'][1668]
keywords

,publisher,publisher_categories,author(s),title,publicationYear,keywords,tot_appearances,tot_pubYrs,tot_authors,tot_publishers
0,dryad,general,ching-ho chang;amanda m. larracuente; rochester,data from: heterochromatin-enriched assemblies...,2018,drosophila melanogaster genome,1,1,1,1
2,dryad,general,ching-ho chang;amanda m. larracuente; rochester,data from: heterochromatin-enriched assemblies...,2018,gene duplications,1,1,1,1
3,dryad,general,ching-ho chang;amanda m. larracuente; rochester,data from: heterochromatin-enriched assemblies...,2018,y chromosome,1,1,1,1
4,dryad,general,ching-ho chang;amanda m. larracuente; rochester,data from: heterochromatin-enriched assemblies...,2018,long read assembly,1,1,1,1
5,dryad,general,ching-ho chang;amanda m. larracuente; rochester,data from: heterochromatin-enriched assemblies...,2018,gene conversion,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
3322,american chemical society,scientific,r. h. schlessinger; yu-jang li,total synthesis of (−)-virginiamycin m<sub>2</...,1996,34 chemical sciences,249,28,249,4
3324,american chemical society,scientific,guillermo c. bazan; george rodriguez; arthur j...,aminoboratabenzene derivatives of zirconium: ...,1996,34 chemical sciences,244,28,244,4
3325,american chemical society,scientific,guillermo c. bazan; george rodriguez; arthur j...,aminoboratabenzene derivatives of zirconium: ...,1996,34 chemical sciences,249,28,249,4
3327,american chemical society,scientific,scott d. cummings; richard eisenberg,tuning the excited-state properties of platinu...,1996,34 chemical sciences,244,28,244,4


In [5]:
# keywords.to_csv("data/keywords.csv")

### Authors
___________________________________
Now we'll create a dataset that filters for authors

In [4]:
authors = df.set_index(['publisher','publisher_categories','keywords', 'title', 'publicationYear']).apply(lambda x: x.str.split(';').explode()).reset_index()
authors = authors.drop_duplicates()
# clean up publisher names

authors = authors[authors["author(s)"].str.contains("http://www.") == False] 
authors = authors[authors["author(s)"].str.contains("rochester") == False] 
# clean up author names
authors['author(s)'] = authors['author(s)'].str.replace('william w. brennessel','brennessel, william w.')
authors['author(s)'] = authors['author(s)'].str.replace('brennessel, w.w.','brennessel, william w.')
authors['author(s)'] = authors['author(s)'].str.replace('jones, w.d.','jones, william d.')
authors['author(s)'] = authors['author(s)'].str.replace('frontier, a.j.','frontier, alison j.')
authors['author(s)'] = authors['author(s)'].str.replace('alison, j. frontier','frontier, alison j.')
authors['author(s)'] = authors['author(s)'].str.replace('richard eisenberg','eisenberg, richard')
authors['author(s)'] = authors['author(s)'].str.replace('eisenberg, r.','eisenberg, richard')
drop = authors['author(s)'] == 'university'
authors = authors[~drop]

#get number of author appearances
authors['tot_appearances'] = authors.groupby('author(s)')['author(s)'].transform('count')
# get every year an author published something
authors['tot_pubYrs'] = authors.groupby(['author(s)'])['publicationYear'].transform('nunique').astype(str)
# get every keyword an author used
authors['tot_keywords'] = authors.groupby(['author(s)'])['keywords'].transform('nunique')
# get every publisher affiliated with the author
authors['tot_publishers'] = authors.groupby(['author(s)'])['publisher'].transform('nunique').astype(str)
authors

,publisher,publisher_categories,keywords,title,publicationYear,author(s),tot_appearances,tot_pubYrs,tot_keywords,tot_publishers
0,dryad,general,"drosophila melanogaster genome, rochester, gen...",data from: heterochromatin-enriched assemblies...,2018,ching-ho chang,3,2,3,3
1,dryad,general,"drosophila melanogaster genome, rochester, gen...",data from: heterochromatin-enriched assemblies...,2018,amanda m. larracuente,1,1,1,1
3,dryad,general,"clinical trial, vaccine, phase 1, adenoviridae...",data from: first-in-human randomized controlle...,2018,kathryn e. stephenson,1,1,1,1
4,dryad,general,"clinical trial, vaccine, phase 1, adenoviridae...",data from: first-in-human randomized controlle...,2018,michael c. keefer,1,1,1,1
5,dryad,general,"clinical trial, vaccine, phase 1, adenoviridae...",data from: first-in-human randomized controlle...,2018,catherine a. bunce,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
10281,american chemical society,scientific,34 chemical sciences; 34 chemical sciences; 34...,aminoboratabenzene derivatives of zirconium: ...,1996,arthur j. ashe,1,1,1,1
10282,american chemical society,scientific,34 chemical sciences; 34 chemical sciences; 34...,aminoboratabenzene derivatives of zirconium: ...,1996,saleem al-ahmad,1,1,1,1
10283,american chemical society,scientific,34 chemical sciences; 34 chemical sciences; 34...,aminoboratabenzene derivatives of zirconium: ...,1996,christian müller,2,2,2,1
10284,american chemical society,scientific,34 chemical sciences; 34 chemical sciences; 34...,tuning the excited-state properties of platinu...,1996,scott d. cummings,1,1,1,1


In [8]:
# authors.to_csv("data/authors.csv")